In [ ]:
# Import required libraries
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                dcc.Dropdown(
                                    id='site-dropdown',
                                    options=[
                                        {'label': 'All Sites', 'value': 'ALL'},
                                        {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                        {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                        {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                        {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                    ],
                                    value='ALL',
                                    searchable=True,
                                    placeholder='Select a Launch Site here'
                                ),

                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div([dcc.Graph(id='success-pie-chart', figure = {})]),
                                html.Br(),

                                html.P("Payload Range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(
                                    id='payload-slider',
                                    min=0,
                                    max=10000,
                                    step=1000,
                                    value=[0, 10000]
                                ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart', figure={})),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(
        component_id='success-pie-chart',
        component_property='figure'
    ),
    Input(
        component_id='site-dropdown',
        component_property='value'
    )
)

def get_pie_chart(entered_site):
    df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(
            df, 
            values='class',
            names='Launch Site',
            title='Number of Sucessful Landings by Site'
        )
    
    else:
        df = df[df['Launch Site'] == entered_site]
        for i in df.index:
            if df.loc[i, 'class'] == 1:
                df.loc[i, 'class'] = 'Sucessful Landings'
            else:
                df.loc[i, 'class'] = 'Failed Landings'

        fig = px.pie(
            data_frame=df,
            names='class',
            title='Percent of Sucessful Landings at: ' + str(entered_site)
        )
    
    return fig

    
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(
        component_id='success-payload-scatter-chart',
        component_property='figure'
    ),
    [Input(
        component_id='site-dropdown', 
        component_property='value'
    ),
    Input(
        component_id='payload-slider',
        component_property='value'
    )]
)

def get_scatter_plot(entered_site, entered_masses):
    df = spacex_df
    df = df[(df['Payload Mass (kg)'] >= entered_masses[0]) & (df['Payload Mass (kg)'] <= entered_masses[1])]
    if entered_site == 'ALL':
        fig = px.scatter(
            df,
            x='Payload Mass (kg)',
            y='class',
            color='Booster Version Category',
            title='Landing Outcome v. Payload Mass'
        )
    else:
        df = df[df['Launch Site'] == entered_site]
        fig = px.scatter(
            df,
            x='Payload Mass (kg)',
            y='class',
            color='Booster Version Category',
            title='Landing Outcome v. Payload Mass at: ' + str(entered_site)
        )
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server()
